In [ ]:
# !pip install transformers
# !pip install sentencepiece

In [ ]:
# Window os
# !pip install "git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf"
# Other os
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [2]:
# 데이터 준비 1
daum_movie_review_df = pd.read_csv('data-files/daum_movie_review.csv')
daum_movie_review_df.head()
daum_X = daum_movie_review_df['review'].to_list()
daum_y = daum_movie_review_df['rating'].map(lambda r: 1 if r > 5 else 0).to_list()
daum_train_X, daum_test_X, daum_train_y, daum_test_y = \
    train_test_split(daum_X, daum_y, stratify=daum_y, random_state=42)
daum_train_X, daum_valid_X, daum_train_y, daum_valid_y = \
    train_test_split(daum_train_X, daum_train_y, stratify=daum_train_y, random_state=42)

In [3]:
# 데이터 준비 2
naver_movie_train_review_df = pd.read_csv('data-files/ratings_train.txt', sep='\t')
naver_movie_test_review_df = pd.read_csv('data-files/ratings_test.txt', sep='\t')

naver_train_X = naver_movie_train_review_df['document'].to_list()
naver_train_y = naver_movie_train_review_df['label'].to_list()
naver_test_X = naver_movie_test_review_df['document'].to_list()
naver_test_y = naver_movie_test_review_df['label'].to_list()
naver_train_X, naver_valid_X, naver_train_y, naver_valid_y = \
    train_test_split(naver_train_X, naver_train_y, stratify=naver_train_y, random_state=42)

In [4]:
# daum_train_X.shape, daum_valid_X.shape, daum_test_X.shape
type(daum_train_X)

list

In [5]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")

daum_train_inputs = tokenizer(daum_train_X, truncation=True, padding=True, return_tensors='pt')
daum_valid_inputs = tokenizer(daum_valid_X, truncation=True, padding=True, return_tensors='pt')
daum_test_inputs = tokenizer(daum_test_X, truncation=True, padding=True, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
print( daum_train_inputs['input_ids'].shape, daum_test_inputs['input_ids'].shape )

for item in daum_train_inputs.items():
    print(item[1][2].shape)
    break

torch.Size([8282, 280]) torch.Size([3682, 289])
torch.Size([280])


In [7]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        pass

    def __getitem__(self, idx):
        input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }
        input['label'] = torch.tensor(self.labels[idx])
        return input
    
    def __len__(self):
        return len(self.labels)
    
daum_train_dataset = ReviewDataset(daum_train_inputs, daum_train_y)
daum_valid_dataset = ReviewDataset(daum_valid_inputs, daum_valid_y)
daum_test_dataset = ReviewDataset(daum_test_inputs, daum_test_y)

In [8]:
for data in daum_train_dataset:
    print(data.keys())
    print(data['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
torch.Size([280])


C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_17896\2483146844.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


In [9]:
device_name = "cuda:0" if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
device

device(type='cuda', index=0)

In [11]:
from transformers import BertModel

In [12]:
class ReviewClassificationModel(nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels): 
        super(ReviewClassificationModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        
        self.pretrained_model = pretrained_model
        self.classifier = nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs)

        bert_clf_token = outputs.last_hidden_state[:,0,:]
        return self.classifier(bert_clf_token)

bert_model = BertModel.from_pretrained("skt/kobert-base-v1")
model = ReviewClassificationModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size) 



config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

loading configuration file config.json from cache at C:\Users\Administrator.User -2023YNCQT\.cache\huggingface\hub\models--skt--kobert-base-v1\snapshots\a9f5849fce18fb088f0cd0f9b29ec3f756958464\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "kobert_version": 1.0,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}



pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\Administrator.User -2023YNCQT\.cache\huggingface\hub\models--skt--kobert-base-v1\snapshots\a9f5849fce18fb088f0cd0f9b29ec3f756958464\pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at skt/kobert-base-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [15]:
from torch.utils.data import DataLoader

daum_train_loader = DataLoader(daum_train_dataset, shuffle=True, batch_size=8)
daum_valid_loader = DataLoader(daum_valid_dataset, shuffle=False, batch_size=8)
daum_test_loader = DataLoader(daum_test_dataset, shuffle=False, batch_size=8)

In [16]:
from transformers import AdamW
import torch.nn.functional as F
import time

model.to(device)
model.train() # 스위치 : 가중치 업데이터 활성화 모드

optim = AdamW(model.parameters(), lr=5e-5) 
criterion = torch.nn.CrossEntropyLoss() 

start = time.time()
num_epochs = 2
for epoch in range(num_epochs):
    total_epoch_loss = 0
    for step, batch in enumerate(daum_train_loader):
        optim.zero_grad() # 기울기 초기화
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'} 
        labels = batch['label'].to(device)
        outputs = model(inputs)

        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float()) # 손실 계산

        if (step+1) % 100 == 0:
            elapsed = time.time() - start
            print(f'Epoch {epoch+1}, batch {step+1}, elapsed time: {elapsed}, loss: {loss}')
        total_epoch_loss += loss
        loss.backward() # 기울기 계산
        optim.step() # 가중치 업데이트
    avg_epoch_loss = total_epoch_loss / len(daum_train_loader)
    print(f'Average loss for epoch {epoch+1}: {avg_epoch_loss}')

c:\ProgramData\miniconda3\envs\study-env2\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_17896\2483146844.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


Epoch 1, batch 100, elapsed time: 374.0636966228485, loss: 0.6320414543151855
Epoch 1, batch 200, elapsed time: 733.6505360603333, loss: 0.5286116600036621
Epoch 1, batch 300, elapsed time: 1099.0544118881226, loss: 0.49474114179611206
Epoch 1, batch 400, elapsed time: 1461.7365765571594, loss: 0.45018264651298523
Epoch 1, batch 500, elapsed time: 1825.981111049652, loss: 0.3763388395309448
Epoch 1, batch 600, elapsed time: 2149.5333375930786, loss: 0.5203408002853394
Epoch 1, batch 700, elapsed time: 2463.477724790573, loss: 0.41436439752578735
Epoch 1, batch 800, elapsed time: 2770.7507317066193, loss: 0.4860530197620392
Epoch 1, batch 900, elapsed time: 3076.558360815048, loss: 0.7401040196418762
Epoch 1, batch 1000, elapsed time: 3380.1996953487396, loss: 0.38239946961402893
Average loss for epoch 1: 0.4486391246318817
Epoch 2, batch 100, elapsed time: 3795.690288066864, loss: 0.22836554050445557
Epoch 2, batch 200, elapsed time: 4109.535093784332, loss: 0.10136257857084274
Epoch 2